In [1]:
import cvxpy as cp

In [2]:
x = cp.Variable()
a = cp.Parameter(nonneg=True)

In [3]:
print('Curvarture of x:', x.curvature)
print('Curvature of a:', a.curvature)
print('curvature of square(x):', cp.square(x).curvature)
print('curvature of sqrt(x):', cp.sqrt(x).curvature)

Curvarture of x: AFFINE
Curvature of a: CONSTANT
curvature of square(x): CONVEX
curvature of sqrt(x): CONCAVE


In [4]:
x = cp.Variable()
y = cp.Variable(pos=True)

In [5]:
x2 = cp.power(x, 2)
w = cp.quad_over_lin(x2,y)
z = 1 + w
f = cp.sqrt(z)

In [6]:
print('curvature of x2:', x2.curvature)
print('curvature of w:', w.curvature)
print('curvature of z:', z.curvature)
print('curvature of f:', f.curvature)

curvature of x2: CONVEX
curvature of w: CONVEX
curvature of z: CONVEX
curvature of f: QUASICONVEX


In [7]:
cp.sqrt(1 + cp.quad_over_lin(cp.square(x),y)).curvature

'QUASICONVEX'

In [8]:
cp.quad_over_lin(x,y).curvature

'CONVEX'

In [9]:
x = cp.Variable(nonneg=True)
fa = cp.minimum(2, x, cp.sqrt(x))
fa.curvature

'CONCAVE'

In [10]:
x = cp.Variable()
fb = cp.power(x, 3)
fb.curvature

'CONVEX'

In [11]:
x = cp.Variable(pos=True)
fc = cp.power(x, 3)
fc.curvature

'CONVEX'

In [12]:
x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)

fd = cp.sqrt(x*cp.minimum(y,2))

fd.curvature

'QUASICONCAVE'

In [13]:
x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)

fe = cp.square(cp.sqrt(x) + cp.sqrt(y))

fe.curvature

'QUASICONCAVE'

**4.3** *Formulating constraints in CVX*. Below we give several convex constraints on scalar variables $x$, $y$, and $z$. Express each one as a set of valid constraints in CVX*. Directly expressing them in CVX* will lead to invalid constraints. You can also introduce additional variables, if needed.

Check your reformulations by creating a small problem that includes these constraints, and solving it using CVX*. Your test problem doesn't have to be feasible, it's enough to verify that CVX* processes your constraints without error.

**(a)** $1/x + 1/y \leq 1, x \geq 0, y \geq 0$

In [36]:
x = cp.Variable(nonneg=True)   # x >= 0
y = cp.Variable(nonneg=True)   # y >= 0

# define objective function (minimize 3*x1 + 2*x2)
objective = cp.Minimize(3*x + 2*y)

# define constraints
constraints = [
    cp.inv_pos(x) + cp.inv_pos(y) <= 1
]

# form and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

9.898979198766426

**(b)** $xy \geq 1, x \geq 0, y \geq 0$

In [44]:
x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)

objective = cp.Minimize(3*x + 2*y)

constraints = [
    x >= cp.inv_pos(y)
]

problem = cp.Problem(objective, constraints)
problem.solve()

4.89897945484331

**(c)** $(x+y)^2/\sqrt{y} \leq x - y + 5$ with implicit constraint $y \geq 0$

In [78]:
x = cp.Variable()
y = cp.Variable(nonneg=True)
s = cp.Variable(nonneg=True)
t = cp.Variable()

constraints = [
    cp.quad_over_lin(x + y, s) <= t,
    t <= x - y + 5,
    y >= cp.square(s)
]

objective = cp.Minimize(3*x + 2*y)

problem = cp.Problem(objective, constraints)
problem.solve()

-4.908788065336004

**(d)** $x + z \leq 1 + \sqrt{xy - z^2}$ $x \geq 0, y \geq 0$

In [88]:
x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)
z = cp.Variable()

cp.square(x + z - 1).curvature

'CONVEX'

In [ ]:
x = cp.Variable(nonneg=True)
y = cp.Variable(nonneg=True)
z = cp.Variable()
s = cp.Variable()

constraints = [
    # x + z <= 1 + cp.sqrt(x*y - cp.square(z))
    cp.square(x + z - 1) <= x*y - cp.square(z)
]

objective = cp.Minimize(3*x + 2*y)

problem = cp.Problem(objective, constraints)
problem.solve()

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
power(var1254 + var1256 + -1.0, 2.0) <= var1254 @ var1255 + -power(var1256, 2.0) , because the following subexpressions are not:
|--  var1254 @ var1255